In [17]:
import  sys
import random 
import os
from base64 import *
import binascii
import numpy as np
import math


In [2]:
def Key_Generation(size):
    key = os.urandom(size)
    return binascii.hexlify(key)


In [3]:
def Generate_Key_State(key):
    key_string  = str(key)
    key_string = key_string[2:-1]
    ls = []
    for i in range(0,len(key_string),2):
        ls.append(key_string[i] + key_string[i+1])
        
    key_state = np.array(ls).reshape(4,4)
    key_state = np.transpose(key_state)
    return key_state   
    
    

In [4]:
def Generate_subkey(prev_key):
    last_column = np.array(prev_key[:,-1])
    last_column = np.transpose(last_column)
    last_column = np.array(last_column)
    last_column = last_column.reshape(4,1)
    return last_column

In [5]:
round_constant = np.array([[0x01,0x02, 0x04 ,0x08,0x10, 0x20, 0x40, 0x80, 0x1B, 0x36],
                [0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00],
                   [0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00],
                   [0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00],
                  
                 ])

In [6]:
def rotword(last_column):
    rot = last_column[1:]

    rot = np.append(rot,last_column[0])
    rot = np.array(rot)
    rot = rot.reshape(4,1)
    return rot
    
    
    
    
    

In [7]:
s_box_values = [
        [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
        [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
        [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
        [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
        [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
        [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
        [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
        [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
        [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
        [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
        [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
        [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
        [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
        [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
        [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
        [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
]

In [8]:
def s_box_lookup(rotc):
    sbl = []
    for h in rotc:
        sbl.append(hex(s_box_values[int(h[0][0],16)][int(h[0][1],16)]))
        
    sbl = np.array(sbl)
    sbl = sbl.reshape(4,1)
    return sbl    
     

In [583]:
# def Generate_next_key(pkfc, sblc, inx):
#     print(pkfc)
#     print(sblc)
#     r = round_constant[:,0]
#     print(r)
    
#     ls = [[]]
#     for i in range(0,4):
#         c = hex(int(pkfc[i],16) ^ int(sblc[i],16) ^ int(r[i]))
#         ls[j].append(c)
#     for i in range(1,4):
        
        
#     ls = np.array(ls)
#     ls = np.transpose(ls)
#     return ls    
        

In [9]:
def Generate_next_key(pk, sblc, rc):
    r = round_constant[:,rc]
    r = np.array(r)
    r = r.reshape(4,1)
    ls = []
    pkfc = pk[:,0].reshape(4,1)
   




    result = list(hex(int(x[0],16)^int(y[0],16)^int(str(z[0]),16)) for x,y,z in zip(pkfc,sblc,r))
    ls.append(result)
    
    for i in range(1,4):
        pkfc = pk[:,i]
        temp = list(hex(int(x,16)^int(y,16)) for x,y in zip(pkfc,result))
        ls.append(temp)

    
    
    ls = np.array(ls)
    ls = np.transpose(ls)
    return ls
    

    
    
    
    
    
    
    
    

In [10]:
def pad(a):
    x = math.ceil(len(a)/16)
    print("Number of 16 blocks" , x)
    p = len(a)%16
    print("Number of paddings" , p)
    print(16-p)
    if p!= 0:
        a=a.ljust(len(a)+(16-p),'0')

    return a

    

In [11]:
def Generate_Plain_State(a):
    ls = []
    result = []
    for i in range(0,len(a), 4):
        ls.append([hex(ord(a[i])), hex(ord(a[i+1])),hex(ord(a[i+2])),hex(ord(a[i+3]))])

       
    for i in range(0,len(ls),4):
        result.append([[ls[i],ls[i+1],ls[i+2], ls[i+3]]])
    result = np.array(result)
    #result = np.transpose(result)
    r = []
    for i in result:
        temp = np.transpose(i[0])
        r.append(temp)
    
    r = np.array(r)
    return r   
   
        

In [12]:
def Add_Round_Key(key,pt):
    result = []
    print( " Passed key \n", key)
    print("passed plain text \n",pt)
    
    for i in range(0,len(pt)):
        for j in range(0,4):
            kc = key[:,j]
            ptc = pt[i][:,j]
            result.append([hex(int(x,16)^int(y,16)) for x,y in zip(kc,ptc)])
    result = np.array(result)
    
    result = result.reshape(len(pt),4,4)
    rf = []


  
    for i in range(0,len(result)):
        r = np.array(result[i])
        r = np.transpose(r)
        rf.append(r)
        
        
        

        

    return rf
   
            
    
    
    
    
    

                
    
    
    

In [13]:
def sub_byte(state):
    sbl = []
    for i in range(0,4):
        c = state[:,i]
        for j in c:
            if len(j)<4:
                r = 0
                c = int(j[2],16)
            else:
                r = int(j[2],16)
                c = int(j[3],16)
            sbl.append(hex(s_box_values[r][c]))


    sbl = np.array(sbl)
    sbl = sbl.reshape(4,4) 
    sbl = np.transpose(sbl)
        #sbl.append(hex(s_box_values[int(h[0][0],16)][int(h[0][1],16)]))
    print(sbl)    
    return sbl    
    

    
    

In [14]:
def shift_rows(s):
    temp10 = s[1][0]
    s[1][0], s[1][1], s[1][2], s[1][3] = s[1][1], s[1][2], s[1][3], temp10
    
    temp20 = s[2][0]
    temp21 = s[2][1]
    
    s[2][0], s[2][1], s[2][2], s[2][3] = s[2][2], s[2][3], temp20, temp21
    
    temp30 = s[3][0]
    temp31 = s[3][1]
    temp32 = s[3][2]
    s[3][0], s[3][1], s[3][2], s[3][3] = s[3][3],temp30,temp31, temp32
    return s

In [47]:
def gmul(a, b):
    if b == 1:
        return a
    tmp = (a << 1) & 0xff
    if b == 2:
        return tmp if a < 128 else tmp ^ 0x1b
    if b == 3:
        return gmul(a, 2) ^ a

In [48]:
def mix_column(state):
    mult_matrix = np.array([
                   ['0x02', '0x03', '0x01', '0x01'],
                   ['0x01', '0x02','0x03', '0x01'],
                   ['0x01', '0x01', '0x02', '0x03'],
                   ['0x03', '0x01', '0x01', '0x02'],
                   ])
    mc = []
    for j in range(0,4):
        c = state[:,j]
        c = c.tolist()
        print("c \n" , c)
        for i in range(0,4):
            m = mult_matrix[i,:]
            #print("c \n",c)
            print("m \n",m)
            m = m.tolist()
            t1 = []
            for k in c:
                t1.append(int(k,16))
            t2 = []    
            for k in m:
                t2.append(int(k,16))
            mc.append(hex( gmul(t1[0],t2[0]) ^ gmul(t1[1],t2[1]) ^ gmul(t1[2],t2[2]) ^ gmul(t1[3],t2[3])))    
            
                
                
            
            
            

            #mc.append(hex( (int(c[0],16)*m[0]) ^ (int(c[1],16)*m[1]) ^ (int(c[2],16)*m[2]) ^ (int(c[3],16)*m[3])))
    mc = np.array(mc)
    mc = mc.reshape(4,4)
    mc = np.transpose(mc)
    
    print(mc)  
    return mc
        
        
    
        
    

In [49]:
k = Key_Generation(16)
print(" Key_Generation \n" + str(k))
print("\n")
ks = Generate_Key_State(k)
print("Generate_Key_State \n"+str(ks))
print("\n")
lc = Generate_subkey(ks)
print(" Generate_subkey \n"+str(lc))
print("\n")
r = rotword(lc)
print(" rotword \n"+str(r))
print("\n")
sbl = s_box_lookup(r)
print("s box lookup \n" + str(sbl))
print("\n")
nxt = Generate_next_key(ks,sbl,0)
print(" Generate_next_key \n" + str(nxt))
print("\n")
a = pad("abcdefghijklmnopqrstuvwxyzzzzz")
print("Padded string " ,a)
print("\n")
c = Generate_Plain_State(a)

print("Generated plain text state \n ", c)
print("\n")

rc = Add_Round_Key(ks,c)
print("Add Round Key \n" , rc)
print("\n")

sb1 = sub_byte(rc[0])
print("sub byte \n" , sb1)
print("\n")

sf = shift_rows(sb1)
print("shift  \n" , sf)
print("\n")
temp = np.array([
        ['0xd4','0xc9','0x72','0x4d'],
        ['0xf2','0x67','0x21','0x63'],
        ['0x7d','0x10','0xc9','0x67'],
        ['0xda','0x72','0xf0','0x72'],
       ])
mc = mix_column(temp)
print("Mix Column \n", mc)
print("\n")


 Key_Generation 
b'382b094d9fc4f298ef9899260b2c078d'


Generate_Key_State 
[['38' '9f' 'ef' '0b']
 ['2b' 'c4' '98' '2c']
 ['09' 'f2' '99' '07']
 ['4d' '98' '26' '8d']]


 Generate_subkey 
[['0b']
 ['2c']
 ['07']
 ['8d']]


 rotword 
[['2c']
 ['07']
 ['8d']
 ['0b']]


s box lookup 
[['0x71']
 ['0xc5']
 ['0x5d']
 ['0x2b']]


 Generate_next_key 
[['0x48' '0xd7' '0xa7' '0x43']
 ['0xee' '0x2a' '0x76' '0xc2']
 ['0x54' '0xa6' '0xcd' '0x53']
 ['0x66' '0xfe' '0x40' '0xeb']]


Number of 16 blocks 2
Number of paddings 14
2
Padded string  abcdefghijklmnopqrstuvwxyzzzzz00


Generated plain text state 
  [[['0x61' '0x65' '0x69' '0x6d']
  ['0x62' '0x66' '0x6a' '0x6e']
  ['0x63' '0x67' '0x6b' '0x6f']
  ['0x64' '0x68' '0x6c' '0x70']]

 [['0x71' '0x75' '0x79' '0x7a']
  ['0x72' '0x76' '0x7a' '0x7a']
  ['0x73' '0x77' '0x7a' '0x30']
  ['0x74' '0x78' '0x7a' '0x30']]]


 Passed key 
 [['38' '9f' 'ef' '0b']
 ['2b' 'c4' '98' '2c']
 ['09' 'f2' '99' '07']
 ['4d' '98' '26' '8d']]
passed plain text 
 [[['0x61' '0x

In [34]:
g = ['0xd4', '0xf2', '0x7d', '0xda']
h = ['0x01','0x02' ,'0x03', '0x01']
s = []
for x,y in zip(g,h):
    x = int(x,16)
    y = int(y,16)
    s.append(x*y)
 

In [36]:
print((s[0] ^ s[1] ^ s[2] ^ s[3]))

157


In [705]:
hex(int('0x3f',16)^int('0x63',16)^int('0x0',16))

'0x5c'

In [431]:
round_constant

[[1, 2, 4, 8, 16, 32, 64, 128, 27, 54],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [434]:
def rotword(last_column):
    rot = last_column[1:]
    rot.append(last_column[0])
    rot = np.array(rot)
    return rot
    
    
    
    
    

In [484]:
l = [0xd3, 0x23]
l = [int(i) for i in l]
print(l)

[211, 35]


In [414]:
Generate_subkey

<function __main__.Generate_subkey(prev_key)>

In [377]:
key = Key_Generation(16)
print(key)
s = str(key)
s = s[2:-1]
ls = []
print(s)
for i in range(0,len(s),2):
    print(s[i] + s[i+1])
    ls.append(s[i] + s[i+1])
ls
res = np.array(ls).reshape(4,4)
print(res)
res = np.transpose(res)
print(res)

b'4ec43f5c2549b6d2f045a7c48b9a6405'
4ec43f5c2549b6d2f045a7c48b9a6405
4e
c4
3f
5c
25
49
b6
d2
f0
45
a7
c4
8b
9a
64
05
[['4e' 'c4' '3f' '5c']
 ['25' '49' 'b6' 'd2']
 ['f0' '45' 'a7' 'c4']
 ['8b' '9a' '64' '05']]
[['4e' '25' 'f0' '8b']
 ['c4' '49' '45' '9a']
 ['3f' 'b6' 'a7' '64']
 ['5c' 'd2' 'c4' '05']]


In [376]:
a = ""
for i in range(8):
    a = a + chr(random.randint(33,127))
print(a)  
b = bytes(a,"utf-8")
print(b)
print(bytes(16))

   

_`3a<|F
b'_\x7f`3a<|F'
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [127]:
rnd = os.urandom(16)
rnd
c= str(os.urandom(20))
b=b64encode(rnd).decode('utf-8')
print(b)
print(c)

NV4IwzH+qV9A/09RVYB2Hw==
b'X-\xb4\x1bm\xfaE\x07\xee\xbf\xca\xee\x0c0]\x81\xb8\x16\x85n'


In [58]:
import math
a = "My Name Is Prathamesh Ravindra Tayade I am 21 Year"

print(len(a))

x = math.ceil(len(a)/16)
print("Number of 16 blocks" , x)

p = len(a)%16
print("Number of paddings" , p)

print(16-p)
if p!= 0:
    a=a.ljust(len(a)+(16-p),'0')
print(a)
len(a)

    
    
    
                
    

50
Number of 16 blocks 4
Number of paddings 2
14
My Name Is Prathamesh Ravindra Tayade I am 21 Year00000000000000


64